In [ ]:
# !pip install pretrainedmodels==0.7.4
# !pip install efficientnet-pytorch==0.6.3
# !pip install timm==0.3.2

In [2]:
!nvidia-smi

import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="3"

import torch
import numpy as np
import cv2
import matplotlib.pyplot as plt
import SimpleITK as sitk
import cv2
from tqdm import tqdm

import sys
sys.path.append(os.path.abspath('/workspace/sunggu'))
sys.path.append(os.path.abspath('/workspace/sunggu/MONAI'))
sys.path.append(os.path.abspath('/workspace/sunggu/4.Dose_img2img'))
sys.path.append(os.path.abspath('/workspace/sunggu/4.Dose_img2img/UGATIT-pytorch-master'))

from sunggu_utils import check_value, take_list, plot_confusion_matrix, list_sort_nicely, find_dir, plot_3D

Wed Apr 21 09:35:24 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 455.45.01    Driver Version: 455.45.01    CUDA Version: 11.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-PCIE...  Off  | 00000000:3B:00.0 Off |                    0 |
| N/A   32C    P0    34W / 250W |  31433MiB / 32510MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  Tesla V100-PCIE...  Off  | 00000000:86:00.0 Off |                    0 |
| N/A   

# Train

In [ ]:
cd /workspace/sunggu/4.Dose_img2img/UGATIT-pytorch-master

In [ ]:
!python main.py --dataset u_got_it --img_size 512 --light True --batch_size 1 --resume True --save_freq

In [ ]:
import functools

In [ ]:
import albumentations as albu

def __scale_width(img, target_width, crop_width, method=1, **kwargs):
    ow, oh = img.shape
    print(ow)
    w = target_width
    h = int(max(target_width * oh / ow, crop_width))
    return albu.functional.resize(img, width=w, height=h, interpolation=method)

downsample_fn = functools.partial(__scale_width, target_width=320, crop_width=320, method=1)

t = albu.Lambda(image=downsample_fn)

In [ ]:
a = np.ones((512,512))

In [ ]:
t(image=a)

# Test

In [3]:
cd /workspace/sunggu/4.Dose_img2img/UGATIT-pytorch-master

/workspace/sunggu/4.Dose_img2img/UGATIT-pytorch-master


In [4]:
!python main.py --dataset u_got_it --phase test --light True --result_dir ./results_off/

number of epochs must be larger than or equal to one

##### Information #####
# light :  True
# dataset :  u_got_it
# batch_size :  1
# iteration per epoch :  1000000

##### Generator #####
# residual blocks :  4

##### Discriminator #####
# discriminator layer :  6

##### Weight #####
# adv_weight :  1
# cycle_weight :  10
# identity_weight :  10
# cam_weight :  1000
 [*] Load FAILURE
 [*] Test finished!


In [ ]:
def diagnose_network(net, name='network'):
    """Calculate and print the mean of average absolute(gradients)
    Parameters:
        net (torch network) -- Torch network
        name (str) -- the name of the network
    """
    mean = 0.0
    count = 0
    for param in net.parameters():
        if param.grad is not None:
            mean += torch.mean(torch.abs(param.grad.data))
            count += 1
    if count > 0:
        mean = mean / count
    print(name)
    print(mean)

In [ ]:
x = torch.rand(10, 10, dtype=torch.complex64)
fft2 = torch.fft.fft2(x)

In [ ]:
fft2.shape

In [ ]:
x

In [ ]:
!pip install torch==1.8.0